In [1]:
import os
import shutil
import glob
from tqdm import tqdm
from multiprocessing import Pool
from typing import List, Tuple
import sqlite3
from langchain.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PyMuPDFLoader,
    TextLoader,
    UnstructuredEmailLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
    PyPDFLoader
)
from langchain.embeddings import HuggingFaceEmbeddings, OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain.vectorstores import FAISS, Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
import yaml
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import importlib
import utils
importlib.reload(utils) 

/Users/selinaydin/PycharmProjects/pythonProject7/utils.py:54: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(


Veritabanındaki toplam vektör sayısı: 0
Veritabanındaki toplam vektör sayısı: 0


<module 'utils' from '/Users/selinaydin/PycharmProjects/pythonProject7/utils.py'>

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
LOADER_MAPPING = {
    ".pds_not": (CSVLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
    ".pdf": (PyMuPDFLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
   
}

def load_single_document(file_path: str) -> List[Document]:
    ext = "." + file_path.rsplit(".", 1)[-1]
    if ext in LOADER_MAPPING:
        loader_class, loader_args = LOADER_MAPPING[ext]
        loader = loader_class(file_path, **loader_args)
        return loader.load()
    raise ValueError(f"Unsupported file extension '{ext}'")


def load_documents_sequentially(source_dir: str, ignored_files: List[str] = []) -> List[Document]:
    """
    Load all documents from the source directory sequentially, ignoring specified files.
    """
    all_files = []
    for ext in LOADER_MAPPING:
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext}"), recursive=True)
        )
    filtered_files = [file_path for file_path in all_files if file_path not in ignored_files]

    results = []
    for file_path in tqdm(filtered_files, desc="Loading documents", ncols=80):
        results.extend(load_single_document(file_path))

    return results

In [4]:
docs = load_documents_sequentially("/Users/selinaydin/PycharmProjects/pythonProject7/source_doc")

Loading documents: 100%|██████████████████████████| 9/9 [00:00<00:00, 10.27it/s]


In [5]:
print(f"Toplam Yüklenen Belgeler: {len(docs)}")
for doc in docs[:5]:  # İlk 5 belgeyi inceleyin
    print(doc.page_content)

Toplam Yüklenen Belgeler: 271
GENEL SORULAR 
 
Ø Hocaların maillerine, telefonlarına nereden ulaşabilirim? 
BademNet sayfasında arama butonundan ulaşabilirsiniz. 
 
Ø Ders programlarını nereden görüntüleyebilirim? 
OBS’de ders programı görüntüleme ekranından ulaşabilirsiniz. 
 
Ø Ders kayıt ekranında alacağım ders açılmamış görünüyor, ne yapabilirim? 
Açılacak derslerin listesi bağlı bulunduğunuz Dekanlık/Yüksekokuldan Öğrenci İşleri Müdürlüğüne 
iletilir. OBS’de ders açılmadı uyarısı veriyor ise ve bu bir zorunlu ders ise mutlaka danışman 
hocanıza veya Dekanlığınıza bu konuyu iletmelisiniz. Danışman hocalarınız bağlı bulunduğunuz 
Fakülte/ Yüksekokul Sekreterliğiniz ile iletişime geçecektir, Öğrenci İşleri Müdürlüğüne bilgi 
verecektir. 
 
Ø Ders kayıt yenilemek istiyorum, nasıl yapabilirim? 
Kayıt yenilemek için Mali İşler Müdürlüğü ile görüşmelisiniz. Ödeme işlemlerini tamamladıktan 
sonra ders kayıt zamanında mutlaka ders kaydınızı tamamlamalısınız.  
Ø Kayıt yenilemeden ders kayı

In [6]:
print(os.path.exists("/Users/selinaydin/PycharmProjects/pythonProject7/source_doc"))  
print(os.listdir("/Users/selinaydin/PycharmProjects/pythonProject7/source_doc"))     

True
['Sıkça Sorulan Sorular-Genel sorular.pdf', '.DS_Store', 'acupep-2023-2024-ogrenci-el-kitabi.pdf', 'yatay-gecis-yonergesi (1).pdf', 'acu-taban-puanlar-.pdf', '2024-2025-guz-yariyili-kurumlararasi-yatay-gecis-kontenjanlari.pdf', 'SIKÇA SORULAN SORULAR.pdf', 'Lisans Programları.pdf', 'ngilizce-hazirlik-programi-ile-ingilizce-dersleri-egitim-ogretim-ve-sinav-yonergesi.pdf', 'birlesik_dosya.pdf']


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(docs)
unique_chunks = list({chunk.page_content: chunk for chunk in chunks}.values())

In [8]:
unique_chunks = list({chunk.page_content: chunk for chunk in chunks}.values())
print(f"Benzersiz parça sayısı: {len(unique_chunks)}")

Benzersiz parça sayısı: 5289


In [9]:
import re

cleaned_chunks = []
for chunk in unique_chunks:
    # Satır başlarını kaldır ve tek satır hale getir
    text_no_newlines = chunk.page_content.replace("\n", " ")
    
    # Fazla boşlukları temizle
    text_no_extra_spaces = ' '.join(text_no_newlines.split())
    
    # Özel karakterleri temizle (sadece harf, sayı ve boşluk bırak)
    cleaned_text = re.sub(r'[^\w\sçÇğĞıİöÖşŞüÜ]', '', text_no_extra_spaces)
    
    # Temizlenmiş metni chunk içine tekrar yaz
    chunk.page_content = cleaned_text
    
    # Temizlenmiş chunk'ı listeye ekle
    cleaned_chunks.append(chunk)

print(f"Benzersiz ve temizlenmiş parça sayısı: {len(cleaned_chunks)}")

Benzersiz ve temizlenmiş parça sayısı: 5289


In [10]:
for chunk in chunks[:10]:  
    print(chunk.page_content.strip()) 

GENEL SORULAR Ø Hocaların maillerine telefonlarına nereden ulaşabilirim BademNet sayfasında arama butonundan ulaşabilirsiniz Ø Ders programlarını nereden görüntüleyebilirim
Ø Ders programlarını nereden görüntüleyebilirim OBSde ders programı görüntüleme ekranından ulaşabilirsiniz Ø Ders kayıt ekranında alacağım ders açılmamış görünüyor ne yapabilirim
Açılacak derslerin listesi bağlı bulunduğunuz DekanlıkYüksekokuldan Öğrenci İşleri Müdürlüğüne iletilir OBSde ders açılmadı uyarısı veriyor ise ve bu bir zorunlu ders ise mutlaka danışman
hocanıza veya Dekanlığınıza bu konuyu iletmelisiniz Danışman hocalarınız bağlı bulunduğunuz Fakülte Yüksekokul Sekreterliğiniz ile iletişime geçecektir Öğrenci İşleri Müdürlüğüne bilgi
verecektir Ø Ders kayıt yenilemek istiyorum nasıl yapabilirim Kayıt yenilemek için Mali İşler Müdürlüğü ile görüşmelisiniz Ödeme işlemlerini tamamladıktan
sonra ders kayıt zamanında mutlaka ders kaydınızı tamamlamalısınız Ø Kayıt yenilemeden ders kayıt yapabiliyor muyum
Ücre

In [11]:
print(f"Toplam Parça Sayısı: {len(chunks)}")
for chunk in chunks[10:25]:  
    print(chunk.page_content)

Toplam Parça Sayısı: 5434
Ø Seçmek istediğim ACU Üniversite Seçmeli dersin kontenjanları dolu seçemiyorum seçmek istiyorum Nasıl arttırabilirim
istiyorum Nasıl arttırabilirim ACU Üniversite Seçmeli derslerinin kontenjanları dersin ilgili öğretim elemanı tarafından
belirlenir Kontenjan arttırımı talep etmeden önce ders programınıza uygun başka bir seçmeli ders seçmelisiniz Tüm kontenjanların dolması halinde dersin ilgili öğretim elemanından
kontenjan artırımını talep edebilirsiniz Eposta adreslerine BademNetten ulaşabilirsiniz Hocanıza mail atarken mailde bilgi kısmına Öğrenci İşlerini eklemeyi unutmayınız
Ø Ders kaydımı yaptım ancak danışmanım onaylamadı ne yapabilirim Ders kaydınızı tamamladıktan sonra danışman hocanıza bildirebilirsiniz
Ø OBSde ders kaydım görünüyor ancak EYSde görünmüyor ne yapabilirim Ders kaydınız sizin ve danışmanınız tarafından onaylandı ise 1 iş günü sonrasında EYSye de
yansıyacaktır Yansımaması halinde Eğitim Teknolojileri Birimine EgitimTeknolojileriacibademe

In [12]:
from utils import load_chroma_db

def create_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-bert-base-dot-v5")
embedding_model = create_embeddings()

chroma_db = load_chroma_db()
print("Chroma veritabanı başarıyla oluşturuldu.")

Chroma veritabanı başarıyla oluşturuldu.


In [13]:
llm = ChatOllama(
    model="llama3.2", 
    temperature=0.4,
    max_tokens=524
)

In [14]:
def create_prompt_template():
    prompt_template = """
   İnsan: Aşağıdaki bağlamı kullanarak soruya detaylı bir şekilde ve anlamlı bir şekilde yanıt ver. Eğer cevabı bilmiyorsan, bunu belirt ve uydurma. Cevabında sadece Türkçe dilini kullan, sana verdiğim veriler dışında herhangi bir şey ekleme veya çıkarma yapma.Türkçe olmayan kelimeler, harfler kullanma. Lütfen verilen bilgileri tam anlamıyla yorumla ve sadece o bilgileri kullanarak yanıt ver. Cevaplarını lütfen sadece sana verilen bilgiler ile ver. Ekstra kendin bir şey katma çünkü resmi bir kurum için resmi bir dil ile doğru bir şekilde yanıtlaman gerekiyor. Sana verilen veri dışına çıkmadan yanıtlaman lazım. Okul kartım kayboldu ne yapabilirim gibi sorulara düz sana soru-cevaplarda ne verilmiş ise onu yaz ekstra bir şey katma. Soruyu anlamadıysan ya da bilmiyorsan bilmediğini ya da anlamadığını belirt ve cevaplama. Soru soran insanları yanlış yönlendiremeyiz, önemli sorular soruyorlar ve doğru cevap almaları lazım. Sana verilen verilerde zaten kesin cümleler ve içerikler var bunların dışına çıkman yanlış olur. Lütfen bunlara dikkat ederek cevapla soruları. Başka okulara göre cevap verme lütfen. Sadece Acıbadem Üniversitesi bazında doğru cevaplar vermen gerekiyor. Okulumuzun EYS, OBS ve BADEMNET platformları var. Buralardan bilgilere ulaşıyoruz. Sadece sana verilen veriler doğrultusunda hareket et. Sana verilen pdf dosyalarına göre cevaplaman lazım yoksa verdiğin bilgiler yanlış olacak ve yanlış yönlendireceğiz okuldaki öğrencileri. Sen arkadaş canlısı Acıbadem Üniversitesi Öğrenci İşleri Asistanısın.

<bağlam>
{context}
</bağlam>

Soru: {question}

Yanıtın aşağıdaki gibi olmalı:
- **Kapsayıcı Olmalı**: Soruyu tamamen anlamalı ve verilen bağlama dayanarak en doğru cevabı üretmelisin.
- **Kesin Olmalı**: Verdiğim bağlamda yer alan herhangi bir bilgiyi değiştirmemeli veya eksik belirtmemelisin.
- **Türkçe Olmalı**: Tüm yanıtların Türkçe dilinde olmalı. Yanıttaki dilin, bağlama uygun olmalı ve Türkçeye sadık kalmalı.
- **Bağlama Sadık Olmalı**: Bağlamda verilen veriler dışına çıkmamalısın. Soruyu yanıtlarken yalnızca verilen bağlamı kullanmalı ve herhangi bir spekülasyon yapmamalısın.

Cevabında aşağıdaki öğeleri kullanarak, anlamlı ve ayrıntılı bir yanıt oluştur:
- Önceki bilgilerden çıkarımlar yaparak, daha önce verilmiş olan verilerle tutarlı ve mantıklı bir yanıt oluştur.
- Eğer verilen bilgi eksikse veya yanlışsa, bunu nazikçe belirterek, kesinlikle doğru olmayan bir bilgi eklememelisin.
-Lütfen sana verilmeyen herhangi bir bilgiyi sallama.
-Sana verilen bilgiye yönelik cevaplar ver ve sallama,uydurma ya da kendine göre cevaplama.Okul ile ilgili sana ne verildiyse onu cevapla

Asistan:
    """
    return PromptTemplate(template=prompt_template, input_variables=["context", "question"])

PROMPT = create_prompt_template()


qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)


def generate_response_with_context(query):
    
    retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 3})
    relevant_docs = retriever.invoke(query)

    
    threshold = 0.9
    filtered_results = [
        doc for doc in relevant_docs
        if doc.metadata.get("similarity", 1) >= threshold
    ]

    
    context = " ".join([doc.page_content for doc in filtered_results[:2]])

    
    result = qa.invoke({"context": context, "query": query})

    return {
        "response": result.get("result", "Yanıt bulunamadı."),
        "context_used": context,
        "source_documents": [doc.page_content for doc in filtered_results[:2]]
    }


query = "Okul kartımı kaybettim ne yapabilirim?"
response = generate_response_with_context(query)


print("\nYanıt:")
print(response["response"])

print("\nKullanılan Bağlam:")
print(response["context_used"])




Yanıt:
Merhaba, Acıbadem Üniversitesi Öğrenci İşleri Asistanısın. Okul kartınızı kaybettikten sonra ne yapabilirim sorusunu sormak istediniz. EYS, OBS ve BADEMNET platformlarına bağlı olarak okul kartınızı kaybeden öğrenciler, okulun Öğrenci İşleri departmentuna ulaşarak durumları bildirmelidir.

Öncelikle, school cardı kaybolduğunuzda, immediate olarak okulun Öğrenci İşleri departmentına ulaşmalısınız. Bu, school cardı kaybolduğunuzda, okul tarafından size daha sonra bir school cardı verilebileceği anlamına gelmektedir. School cardı kaybedilen öğrenciler, okulun Öğrenci İşleri departmentuna ulaşarak, school cardı kaybolma durumunu bildirmelidir.

Önceki bilgilerden çıkarımlar yaparak, school kartınızı kaybettikten sonra en uygun yöntemler neler olduğu hakkında bilgi alabiliriz. School kartınızı kaybeden öğrenciler, okulun Öğrenci İşleri departmentına ulaşarak, school cardı kaybolma durumunu bildirmelidir. School cardı kaybedilen öğrenciler, okulun Öğrenci İşleri departmentuna ulaşara

In [15]:
generate_response_with_context("Hazırlık Güz dönemi final sınavları ne zaman?")

{'response': 'Hazırlık Güz Dönemi final sınavları 20-25 Ekim 2023 tarihleri arasında Acıbadem Üniversitesi tarafından organizasyon yapılmıştır. Bu tarihlerde öğrenciler, respective platformlardan (EYS, OBS ve BADEMNET) yapılan hazırlık workşenlerinden sonra final sınavlarına katılabileceklerdir.',
 'context_used': '',
 'source_documents': []}

In [33]:
generate_response_with_context("okula otobüsle nasıl gidebilirim ?")

{'response': "Merhaba! Okula otobüsle nasıl gidebilirim sorusuna respuesta olarak, Acıbadem Üniversitesi Öğrenci İşleri Asistanısın. \n\nÖncelikle, okulumuzun EYS platformu üzerinden otobüs bilet kaufen optionu mevcuttur. Bu platformda, öğrencilerin otobüs biletlerini online olarak satın alabilmeleri mümkün. Bu sayede, öğrenciler otobüsle okula girmek için online bilet purchase yapabilirler.\n\nAyrıca, OBS ve BADEMNET platformlarından da bilgilere ulaşabilirsiniz. Bu platformlar student card ile otobüs biletleri satın alabilirsiniz. Öğrencilerin student card'lerini online olarak kullanmaları mümkün. \n\nÖzetle, okulumuzun EYS platformu üzerinden online bilet purchase yapabilir ve otobüsle okula gidebilirsiniz.",
 'context_used': '',
 'source_documents': []}

In [34]:
generate_response_with_context("acıbadem üniversitesi önlisans bölümleri nelerdir ?")

{'response': 'Acıbadem Üniversitesi Önlisans bölümleri:\n\n- Edebiyat\n- İktisat\n- Mühendislik\n- Fen Bilimleri\n- Hukuk',
 'context_used': '',
 'source_documents': []}